In [ ]:
!pip install neo4j python-dotenv pandas pyvis pydantic fastapi python-multipart matplotlib pandas

In [ ]:
pip install  uvicorn["standard"]

In [2]:
import os
os.environ["NEO4J_DB"]= "neo4j"
NEO4J_DB = os.getenv("NEO4J_DB")

In [3]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "bolt://localhost:7687"
AUTH = (NEO4J_DB, "password")

driver =  GraphDatabase.driver(URI, auth=AUTH, database=NEO4J_DB) # explicit db allows the driver to work more efficiently
try:
    driver.verify_connectivity()
    print("Connection established.")
except:
    print("Connection failed.")

Connection established.


_____

## Project Activity Schema

In [4]:
from pydantic import BaseModel, Field
from enum import Enum
from typing import Optional, Union
from datetime import datetime

class ActivityStatusEnum(str, Enum):
    not_started = "not-started"
    in_progress = "in-progress"
    completed = "completed"

# Enum to represent different types of activity-to-activity relationships
class RelationEnum(str, Enum):
    finish_to_start = "finish-to-start"  # A dependent activity cannot start until its predecessor finishes (default)
    start_to_start = "start-to-start"  # A dependent activity cannot start until the predecessor has started
    finish_to_finish = "finish-to-finish"  # A dependent activity cannot finish until its predecessor finishes
    start_to_finish = "start-to-finish"  # A dependent activity cannot finish until its predecessor starts

# Model to represent an activity
class Activity(BaseModel):
    early_start: Optional[int] = Field(None, description="The earliest time the activity can start")
    early_finish: Optional[int] = Field(None, description="The earliest time the activity can finish")
    late_start: Optional[int] = Field(None, description="The latest time the activity can start without delaying the project")
    late_finish: Optional[int] = Field(None, description="The latest time the activity can finish without delaying the project")
    duration: Optional[int] = Field(..., gt=0, description="The duration of the activity in time units")
    name: Optional[str] = Field(None, description="The name of the activity")
    description: Optional[str] = Field(None, description="A detailed description of the activity")
    total_float: Optional[int] = Field(None, description="The total float of the activity, indicating its flexibility")
    free_float: Optional[int] = Field(None, description="The free float of the activity, indicating flexibility without affecting successors")
    start_date: Optional[datetime] = Field(None, description="The date the activity is scheduled to start")
    end_date: Optional[datetime] = Field(None, description="The date the activity is scheduled to finish")
    status: Optional[ActivityStatusEnum] = Field(None, description="The status of the activity")
    # TODO: Extend to include time and resource consumption details

# Model to represent a predecessor activity
class Predecessor(BaseModel):
    id: Union[str, int] = Field(..., description="A unique identifier for the predecessor activity")
    early_start: Optional[int] = Field(None, description="The earliest start time of the predecessor activity")
    early_finish: Optional[int] = Field(None, description="The earliest finish time of the predecessor activity")

# Model to define relationships between activities
class ActivityToActivityRel(BaseModel):
    activity: Predecessor = Field(..., description="The predecessor activity involved in the relationship")
    type: RelationEnum = Field(default=RelationEnum.finish_to_start, description="The type of relationship between activities")
    duration: Optional[int] = Field(default=0, ge=0, description="Lead or lag time for the relationship, must be non-negative")


## Resource Allocaion Schema
We can think of introducing geofenced cost of resources like how it is done in products. This data will be used in price analysis for BoQs

### Labour

In [5]:
from enum import Enum
from typing import Optional, List
from pydantic import BaseModel, Field

# Enums for predefined choices
class SkillEnum(str, Enum):
    plumbing = "plumbing"
    masonry = "masonry"
    carpentry = "carpentry"  
    electrical = "electrical"

class RateEnum(str, Enum):
    perHour = "per-hour"
    perDay = "per-day"
    perTask = "per-task" 

class GroupEnum(str, Enum):
    admin = "admin"
    supervisor = "supervisor"
    worker = "worker"

# Job class to describe roles
class Job(BaseModel):
    title: str = Field(..., description="Job title, e.g., construction manager, resident engineer")
    description: Optional[str] = Field(None, description="Optional description of the job role")

# Skill class to define a skill and its associated rate
class Skill(BaseModel):
    type: SkillEnum = Field(..., description="Type of skill, e.g., plumbing, masonry")
    rate: RateEnum = Field(..., description="Rate type, e.g., per-hour, per-day")
    remuneration: float = Field(..., gt=0, description="Remuneration value must be greater than 0")

# Employee class for system authentication and user information
class Employee(BaseModel):
    name: str = Field(..., description="Full name of the employee")
    password: Optional[str] = Field(None, description="Optional password for system authentication")
    username: Optional[str] = Field(None, description="Optional unique username for the employee")
    group: Optional[GroupEnum] = Field(None, description="Employee group classification")
    job: Optional[Job] = Field(None, description="Job details for the employee")
    skills: Optional[List[Skill]] = Field(default_factory=list, description="List of skills possessed by the employee")

# EmployeeSkill class to link an employee to a skill
class EmployeeSkill(BaseModel):
    skill: Skill = Field(..., description="The skill the employee possesses")
    employee: Employee = Field(..., description="The employee associated with this skill")

# Labor class for representing available labor and their associated skill
class Labor(BaseModel):
    type: SkillEnum = Field(..., description="The type of labor skill")
    employee: Employee = Field(..., description="The employee performing this labor task")


### Equiment

In [6]:
from pydantic import BaseModel, Field
from enum import Enum

# Enum to represent equipment sourcing options
class EquipmentSourceEnum(str, Enum):
    purchase = "purchase"  # Buying the equipment outright
    leasing = "leasing"  # Leasing for a period, may include a purchase option
    renting = "renting"  # Renting for temporary use, availability-dependent

# Model to represent equipment details
class Equipment(BaseModel):
    name: str = Field(..., description="The name of the equipment")
    description: str = Field(..., description="A detailed description of the equipment")
    source: EquipmentSourceEnum = Field(..., description="The sourcing method for the equipment")
    rate: RateEnum = Field(..., description="The rate type for the equipment (per hour or per day)")
    cost: float = Field(..., gt=0, description="The base cost of the equipment usage")
    transport_cost: float = Field(0, ge=0, description="The cost of transporting the equipment, default is 0")


### Raw Material
Here is where Jumba ecosystem is integrated. Depending on requirements of the project Jumba will be used as a procurement partner since it handles material sourcing and logistics.

In [7]:
class Material(BaseModel):
    name: str
    description: str # include the variance e.g., if cement what type rapid curing or normal curing
    quantity: int
    unitCost: int  # geofenced price

### Resource node

In [8]:
class ResourceEnum(str, Enum):
    equipment = "equipment"
    material = "material"
    labor = "labor"

class Resource(BaseModel):
    type: ResourceEnum
    equipment: Optional[Equipment]
    material: Optional[Material]
    labor: Optional[Labor]

### Allocation node

In [9]:
class AllocationStatusEnum(str, Enum):
    pending = "pending"
    allocated = "allocated"
    released = "released"

class Allocation(BaseModel):
    activityID: Optional[Union [int, str]]
    resourceID: Optional[Union [int, str]]
    quantity: Optional[int]
    unit_cost: Optional[float]
    total_cost: Optional[float]
    allocation_date: Optional[datetime]
    release_date: Optional[datetime]
    status: AllocationStatusEnum = "pending"

class ResourceRequestStatusEnum(str, Enum):
    pending = "pending"
    fulfilled = "fulfilled"
    deficit = "deficit"

class ResourceRequest(BaseModel):
    resource: Union[Material, Equipment, Labor]
    quantity: int
    duration: int
    status: ResourceRequestStatusEnum = "pending"

## Resource Allocation Brain Storming
TODO: decide allocate resource when creating an activity or after all activities have been created
### Goal
- Is the resource available, if not what actions to perform to make it available e.g., alert user, or AI agents to create orders for material or equipment and user just confirms and Jumba Ecosystem handles the rest. (Just In Time production we got here)
- What is the cost of allocating the resource required by the activity. ( information be used in price analysis for BoQ)
- A resouce might not meet the requirement so what is the deficit (User can see impending risks in terms of resources required)

This allocation should be on a timeline where we release resources (the non consumable resource e.g., equipment and labour) once an activity is over so that it is available for the subsequent activities. Advance the resource allocation which have constraints to suggest optimal use of resources to ensure no wastage.

- Each ACTIVITY will have a corresponding ALLOCATION node (store resource allocated metadata i.e., the cost of the allocation depending on geofencing)
- An activity requests for a resouce to this allocation node by a dependency REQUIRES
- Perform DB query to get the resource
    - For materials Jumba's ecosystem will calculate the unitCost, shipping cost and constraints e.g., MoQ for the materials
    - For equipment Jumba can introduce this line of products for leasing, renting or purchasing unitCost and transportation cost
    - Labour on the contractors part a DBMS with the skills available to the contractor and cost of this labour. For manual labour consider introducing an external app like JumbaGo. We will have a labour contractor who adds the skills available to him or her.
- From this allocations will act as saving best practice (a need in civil works for documenting the best practices in implementations)

### Suggested Approach
We want to manage global resources available to the developer then track detailed allocation data of these resources.

#### Structure of the Subgraph

- ACTIVITY - [:REQUIRES] → RESOURCE indicates what an activity needs.
- ACTIVITY - [:ALLOCATED] <- ALLOCATION represents the actual allocation of resources.
- ALLOCATION - [:USES] -> RESOURCE links allocations to specific resources.

#### RESOURCE Node:


#### ALLOCATION Node:
Stores allocation-specific details like quantity, rate, cost, etc.
Relationships:

### Design Overview
#### Nodes
##### ACTIVITY:
Represents a construction or project-related activity.
Attributes: name, duration, start_date, end_date, status, etc.
##### ALLOCATION:
Captures metadata about resource allocation.

Attributes: cost, quantity, unit_cost, transport_cost, allocation_date, release_date, etc.
##### RESOURCE:
Represents the global pool of resources (e.g., labour of type plumbing, equipment of type mixer). 

For material this will be ordered from Jumba. (Create Orders which user will confirm when the time comes upon confirmation Jumba handles the rest)

Attributes depend on the resource type:
- Material: type, unit_cost, MOQ, availability, etc.
- Equipment: type, source, rental_cost, transport_cost, etc.
- Labor: skill, rate, availability, etc.

#### Relationships
##### [:REQUIRES]:
Between ACTIVITY and RESOURCE (via ALLOCATION node).
Captures the dependency of an activity on a specific resource.
##### [:ALLOCATED]:
Between ACTIVITY and ALLOCATION.
Tracks the allocation metadata for resources requested by the activity.
##### [:USES]:
Between ALLOCATION and RESOURCE.
Links allocated resources to the corresponding resource pool.
##### [:RELEASES] (Optional):
Between ALLOCATION and RESOURCE.
Captures the lifecycle of non-consumable resources (e.g., equipment, labor).

** NOTE: In the dashboard we should have who the supplier confirmed to fullfill a resource. **




In [86]:
# Function to create a resource node in the graph database this will be from the developers account
def createResource():
    try:
        with driver.session(database=NEO4J_DB) as session: 
            session.run(
                """
                MERGE (labor:RESOURCE {type: "labor", skill: "plumbing", rate: 50, availability: 10})
                MERGE (equipment:RESOURCE {type: "equipment", name: "Excavator", source: "leasing", rental_cost: 200, transport_cost: 50})
                MERGE (material:RESOURCE {type: "material", name: "Concrete", unit_cost: 20, MOQ: 5, availability: 100})"""
            )
    except Exception as e:
        raise e
createResource()

/var/folders/pq/j682cyjd7cb0xq6g_rj_397w0000gn/T/ipykernel_89373/3363038530.py:4: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database=NEO4J_DB) as session:


In [77]:
# check if allocation is full
"""MATCH (n {name: "Genesis"})-[requires:REQUIRES]->(allocation:ALLOCATION) 
MATCH (n)-[allocated:ALLOCATED]->(allocation:ALLOCATION)
RETURN requires.quantity as requiredQuantity, n.quantity as allocatedQuantity"""

'MATCH (n {name: "Genesis"})-[requires:REQUIRES]->(allocation:ALLOCATION) \nMATCH (n)-[allocated:ALLOCATED]->(allocation:ALLOCATION)\nRETURN requires.quantity as requiredQuantity, n.quantity as allocatedQuantity'

In [78]:
# an activity requires a resource (activity is requesting a resource)
"""CREATE (activity:ACTIVITY) ... RETURN ID(activity) as activityID"""

'CREATE (activity:ACTIVITY) ... RETURN ID(activity) as activityID'

### Allocating Labour

In [79]:
# aget the activity
"""MATCH (activity:ACTIVITY) WHERE ID(activity) = $activityID"""

# from the required resource
# Query DB for the resource (predefined skills the company has) to get the cost of this resource according to geofencing, and if available and amount available
# save it in a variable resource
# update the allocation with the metadata of the resources according to geofencing also constraints of delivering the labour

# create an allocation for the activity if the activity requires resource
"""CREATE (allocation: ALLOCATION {activityID: $activityID})"""
# create a relationship between the activity and allocation
"""(activity)-[requires:REQUIRES {type: labour, name:'plumbing', quantity: 5}]->(allocation)""" # activity expects  ... in the allocation

"""SET allocation.unitCost = $resource.unitCost, allocation.rate = resource.rate  """ # required.quantity x allocaition.unitCost = price analysis for resource

# What has been allocated depending the resource constraints e.g., available_quantity, 
# with MoQ will result to wastage (manufacturer and developer relationship to get exact quantity required)
"""CREATE (allocation) - [:ALLOCATED {quantity: 5}] -> (acitivity)"""

'CREATE (allocation) - [:ALLOCATED {quantity: 5}] -> (acitivity)'

### Allocating material
Query DB for the materials Jumba has and its cost in terms of geofencing. If not available i.e., the cost is not available aleart someone (Jumba and User)

In [80]:
# aget the activity
"""MATCH (activity:ACTIVITY) WHERE ID(activity) = $activityID"""

# from the required resource
# Query DB for the resource (predefined resources that Jumba can deliver within their procurement) to get the cost of this resource according to geofencing, and if available and amount available
# save it in a variable resource
# update the allocation with the metadata of the resources according to geofencing also constraints of delivering the material e.g., MoQ

# create an allocation for the activity if the activity requires resource
"""CREATE (allocation: ALLOCATION {activityID: $activityID})"""
# create a relationship between the activity and allocation
"""(activity)-[requires:REQUIRES {type: material, name:cement, quantity: 5}]->(allocation)""" # activity expects  ... in the allocation

"""
    SET allocation.unitCost = $resource.unitCost, allocation.moq = resource.moq  """ # required.quantity x allocaition.unitCost = price analysis for resource

# What has been allocated depending the resource constraints e.g., available_quantity, 
# with MoQ will result to wastage (manufacturer and developer relationship to get exact quantity required)
"""CREATE (allocation) - [:ALLOCATED {quantity: 5}] -> (acitivity)"""

'CREATE (allocation) - [:ALLOCATED {quantity: 5}] -> (acitivity)'

### Allocating equipment

In [81]:
# aget the activity
"""MATCH (activity:ACTIVITY) WHERE ID(activity) = $activityID"""

# from the required resource
# Query DB for the resource (predefined resources that Jumba can deliver within their procurement) to get the cost of this resource according to geofencing, and if available and amount available
# save it in a variable resource
# update the allocation with the metadata of the resources according to geofencing and constraints

# create an allocation for the activity if the activity requires resource
"""CREATE (allocation: ALLOCATION {activityID: $activityID})"""
# create a relationship between the activity and allocation
"""(activity)-[requires:REQUIRES {type: equiment, name:crane, quantity: 2}]->(allocation)""" # activity expects  ... in the allocation

"""SET allocation.unitCost = $resource.unitCost, allocation.rate = $allocation.rate """ # required.quantity x allocaition.unitCost = price analysis for resource

"""CREATE (allocation) - [:ALLOCATED {quantity: 1}] -> (acitivity)"""

'CREATE (allocation) - [:ALLOCATED {quantity: 1}] -> (acitivity)'

## Data formatting

In [10]:
import pandas as pd
def formatter(records):
    # Loop through results and do something with them
    for record in records:
        return record.data() # obtain record as dict
    
def to_dataframe(data):
    return pd.DataFrame(data)

def to_csv(data: pd.DataFrame, filename):
    data.to_csv(filename)

def to_excel(data: pd.DataFrame, filename):
    data.to_excel(filename)

In [38]:
def resetGraph():
    try:
        with driver.session(database=NEO4J_DB) as session:
            session.run("""MATCH (n:ACTIVITY) SET n += {earlyStart: NULL, earlyFinish: NULL, lateStart: NULL, lateFinish: NULL, freeFloat: NULL, totalFloat: NULL}""")    
        return
    except Exception as e:
        raise e

## Start Node

Create a start node since we are using activity on node. This will be connecting activities that do not have predecessors

In [ ]:
# create the start node
def initiateProjectSchedule():
    try:
        with driver.session(database=NEO4J_DB) as session:
            results = session.run("""
                MERGE (start:ACTIVITY {name: $name, description: $description, duration: 0, earlyStart: 0, earlyFinish: 0, lateStart: 0, lateFinish: 0})
                RETURN start
            """,
            name = "Genesis",
            description = "This marks the start of your project no resources are consumed"
            )
        return formatter(results)
    except Exception as e:
        print(e)
initiateProjectSchedule()

## Activity Node and Edge
Create activity together with dependencies between each other or start node

In [292]:
# depends on will be the ID of the predecessor activities
def createActivity(activity: Activity, predecessors: list[ActivityToActivityRel] = []):
    
    try:
        with driver.session(database="neo4j") as session:
            if len(predecessors):
                predecessors = [{"activity": vars(obj.activity), "type": obj.type, "duration": obj.duration} for obj in predecessors] # serialize the objects
                # activity has predecessors
                results = session.run("""
                    CREATE (activity:ACTIVITY {name: $name, description: $description, duration: $duration})
                    WITH activity, $predecessors as batch
                    UNWIND batch as dependency
                    MATCH (predecessor:ACTIVITY)
                    WHERE ID(predecessor) = dependency.activity.id
                    CREATE (activity)<-[:DEPENDS_ON {type: dependency.type, duration: dependency.duration}]-(predecessor)""", predecessors=predecessors, name=activity.name, description=activity.description, duration=activity.duration)
                return formatter(results)
            else:
                # Activity has no predecessor start nodes
                results = session.run("""
                        MATCH (genesis:ACTIVITY {name: "Genesis"})
                        CREATE (start:ACTIVITY {name: $name, description: $description, duration: $duration, earlyStart: COALESCE(genesis.earlyFinish, 0), earlyFinish: COALESCE(genesis.earlyFinish, 0) + COALESCE($duration, 0)})
                        CREATE (start)<-[:DEPENDS_ON]-(genesis)
                    """, 
                    name = activity.name,
                    description = activity.description,
                    duration = activity.duration)
                return formatter(results)
    except Exception as e:
        raise e

#### An activity without a dependency
The reason we needed a genesis node

In [8]:
# creating the first activity with no dependecy
createActivity(Activity(name="Excavation", description="It is just digging staff", duration=10))

/var/folders/pq/j682cyjd7cb0xq6g_rj_397w0000gn/T/ipykernel_89373/1068076115.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database="neo4j") as session:


In [9]:
# creating the second activity with no dependecy
createActivity(Activity(name="soil test", description="Confirming strength of soil", duration=3))

/var/folders/pq/j682cyjd7cb0xq6g_rj_397w0000gn/T/ipykernel_89373/1068076115.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database="neo4j") as session:


### Activity with more tha one predecessor

In [ ]:
# has more than one predecessor
predecessors = [
    ActivityToActivityRel(activity=Predecessor(id = 1, earlyStart=0, earlyFinish=10), duration = 0),
    ActivityToActivityRel(activity=Predecessor(id = 2, earlyStart=0, earlyFinish=3), duration = 7),
]
createActivity(Activity(name="Poor foundation", description="placing foundation according to design", duration=2), predecessors=predecessors)

### Activity with 1 predecessor

In [ ]:
# has one predecessor
predecessors = [
    ActivityToActivityRel(activity=Predecessor(id = 3, earlyStart=0, earlyFinish=10), duration = 0),
]
createActivity(Activity(name="Curing", description="cure for 7 days", duration=7), predecessors=predecessors)

### Activity with three predecessors

In [ ]:
# has more than one predecessor
predecessors = [
    ActivityToActivityRel(activity=Predecessor(id = 5, earlyStart=0, earlyFinish=10), duration = 0),
]
createActivity(Activity(name="Inspection", description="Inspect the foundation before back filling", duration=1), predecessors=predecessors)

In [ ]:
# has more than one predecessor
predecessors = [
    ActivityToActivityRel(activity=Predecessor(id = 2, earlyStart=0, earlyFinish=10), duration = 0),
    ActivityToActivityRel(activity=Predecessor(id = 3, earlyStart=0, earlyFinish=10), duration = 0),
    ActivityToActivityRel(activity=Predecessor(id = 4, earlyStart=0, earlyFinish=10), duration = 0),
]
createActivity(Activity(name="Inspection", description="Inspect the foundation before back filling", duration=1), predecessors=predecessors)

## Graph Traversing

For scheduling constraints between the activities or tasks in the graph must be a Directed Acyclic Graph 

TODO: ensure in UI user always creates an asyclic graph.

There is a direction in the dependency of tasks and due to precedence among activities the graph is acyclic.

For this graph we need to have a Topological ordering such that any directed path in the graph traverses the nodes in an increasing order. 

And for this directed graph we may have more than on topological order if graph some parts are not connected.

TODO: Confirm possibility of more than 1 topological ordering of an acyclic graph

### Forward Pass
To calculate early start and early finish
There is precedence within the activities
ESj = Max(EFpredecessors)
EFj = ESj + tj

- get all activity nodes in array
- calculate topological order according to relationship DEPENDS_ON for each activity (might be expensive for large graphs)
- for each activity node get its predecessors
- value of duration of the relationship
- use formula above to get EF and ES of current activity

In [55]:
resetGraph()

This has to be run a couple of times to get the correct values of early Start and early finish

TODO: run the query once for forward pass

In [61]:
def forwardPass():
    try:
        with driver.session(database=NEO4J_DB) as session:
            results = session.run("""
                        // Match all activities and their predecessors
                        MATCH (activity:ACTIVITY)
                        OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)

                        // Collect predecessors for each activity
                        WITH activity, COLLECT(predecessor) AS predecessors

                        // Find the depth of each activity for topological order
                        OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
                        WITH activity, predecessors, 
                            COALESCE(MAX(LENGTH(path)), 0) AS depth, 
                            SIZE(predecessors) AS incomingDependencies

                        // Order activities by depth (descending) and by number of incoming dependencies (ascending) topological order
                        ORDER BY depth DESC, incomingDependencies ASC 

                        // Calculate earlyStart and earlyFinish for each activity
                        WITH COLLECT([activity, predecessors]) AS activityPredecessorsList
                        UNWIND activityPredecessorsList AS activityPredecessorsTuple
                        WITH activityPredecessorsTuple[0] AS currentActivity, 
                            REDUCE(
                                maxFinish = 0, 
                                pred IN activityPredecessorsTuple[1] |                    
                                CASE 
                                    WHEN pred IS NOT NULL AND pred.earlyFinish IS NOT NULL AND pred.earlyFinish > maxFinish 
                                    THEN pred.earlyFinish  
                                    ELSE maxFinish 
                                END
                            ) AS earlyStart
                                  
                        // update early start and  earlyFinish
                        SET currentActivity += {
                            earlyStart: COALESCE(earlyStart, 0), 
                            earlyFinish: COALESCE(earlyStart, 0) + currentActivity.duration
                        }
                                  
                        // Return the activity name, earlyStart, and earlyFinish
                        RETURN currentActivity.name AS activityName, 
                            currentActivity.earlyStart AS earlyStart, 
                            currentActivity.earlyFinish AS earlyFinish
                        ORDER BY currentActivity.depth ASC
                        """)
            return results.data()
    except Exception as e:
        print(e)
        raise e
to_dataframe(forwardPass())

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n                        // Match all activities and their predecessors\n                        MATCH (activity:ACTIVITY)\n                        OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)\n\n                        // Collect predecessors for each activity\n                        WITH activity, COLLECT(predecessor) AS predecessors\n\n                        // Find the depth of each activity for topological order\n                        OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)\n                        WITH activity, predecessors, \n                            COALESCE(MAX(LENGTH(path)), 0) AS depth, \n                           

,activityName,earlyStart,earlyFinish
0,Genesis,0,0
1,A - Wall panel,0,4
2,B,4,7
3,C,4,6
4,D,7,12
5,E,7,10
6,F,12,16
7,G,16,19


### Backward Pass
To calculated latest start and latest finish, this query only if forward pass is correct. The good thing this will run only once since the forward pass has updated values

In [65]:
def backwardPass():
    try:
        with driver.session(database=NEO4J_DB) as session:
            results = session.run("""
                        // match all activities and their successors
                        MATCH (activity:ACTIVITY)
                        OPTIONAL MATCH (activity)-[:DEPENDS_ON]->(successor:ACTIVITY)

                        // collect successors for each activity
                        WITH activity, COLLECT(successor) AS successors

                        // find the depth of each activity for topological order
                        OPTIONAL MATCH path = (activity)<-[:DEPENDS_ON*]-(dependent:ACTIVITY)
                        WITH activity, successors, 
                            COALESCE(MAX(LENGTH(path)), 0) AS depth, 
                            SIZE(successors) AS outgoingDependencies

                        // order activities by depth (descending) and by number of outgoing dependencies (ascending) topological order
                        ORDER BY depth ASC, outgoingDependencies DESC 

                        // calculate lateStart and lateFinish for each activity
                        WITH COLLECT([activity, successors]) AS activitySuccessorsList
                        UNWIND activitySuccessorsList AS activitySuccessorsTuple
                        WITH activitySuccessorsTuple[0] AS currentActivity, activitySuccessorsTuple, activitySuccessorsList
                            WITH currentActivity, activitySuccessorsTuple, activitySuccessorsList, REDUCE(
                                minStart = LAST(activitySuccessorsList)[0].earlyFinish,
                                succ IN activitySuccessorsTuple[1] |                    
                                CASE 
                                    WHEN succ IS NOT NULL AND succ.lateStart IS NOT NULL AND succ.lateStart < minStart 
                                    THEN succ.lateStart  
                                    ELSE minStart 
                                END
                            ) AS lateFinish
                        
                        // update lateStart and lateFinish for each activity
                        SET currentActivity += {
                            lateFinish: COALESCE(lateFinish, currentActivity.earlyFinish), 
                            lateStart: COALESCE(lateFinish, currentActivity.earlyFinish) - currentActivity.duration
                        }

                        // return the results
                        RETURN currentActivity.name AS activityName, 
                            currentActivity.lateStart AS lateStart, 
                            currentActivity.lateFinish AS lateFinish
                        ORDER BY currentActivity.depth DESC""")
            return results.data()
    except Exception as e:
        print(e)
        raise e
to_dataframe(backwardPass())

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n                        // match all activities and their successors\n                        MATCH (activity:ACTIVITY)\n                        OPTIONAL MATCH (activity)-[:DEPENDS_ON]->(successor:ACTIVITY)\n\n                        // collect successors for each activity\n                        WITH activity, COLLECT(successor) AS successors\n\n                        // find the depth of each activity for topological order\n                        OPTIONAL MATCH path = (activity)<-[:DEPENDS_ON*]-(dependent:ACTIVITY)\n                        WITH activity, successors, \n                            COALESCE(MAX(LENGTH(path)), 0) AS depth, \n                            SIZE(succes

,activityName,lateStart,lateFinish
0,Genesis,7,7
1,A - Wall panel,3,7
2,B,4,7
3,C,10,12
4,D,7,12
5,E,13,16
6,F,12,16
7,G,16,19


### Calculating Slack
- Free Slack/float - Amount of time you can delay any given activity without delaying the end date of the entire project
- Total Slack/float - Amount of time you can delay an activity without delaying the earliest start of any subsequent activity

In [66]:
def calculateSlack():
    try:
        with driver.session(database=NEO4J_DB) as session:
            results = session.run("""
                        MATCH (activity:ACTIVITY)
                        OPTIONAL MATCH (activity)-[:DEPENDS_ON]->(successor:ACTIVITY)

                        WITH activity, COLLECT(successor) AS successors

                        OPTIONAL MATCH path = (activity)<-[:DEPENDS_ON*]-(dependent:ACTIVITY)
                        WITH activity, successors, 
                            COALESCE(MAX(LENGTH(path)), 0) AS depth, 
                            SIZE(successors) AS outgoingDependencies

                        ORDER BY depth ASC, outgoingDependencies DESC 

                        WITH COLLECT([activity, successors]) AS activitySuccessorsList
                        UNWIND activitySuccessorsList AS activitySuccessorsTuple
                        WITH activitySuccessorsTuple[0] AS currentActivity, activitySuccessorsTuple
                            WITH currentActivity, activitySuccessorsTuple, REDUCE(
                                minSuccEarlyStart = activitySuccessorsTuple[1][0].earlyStart, 
                                succ IN activitySuccessorsTuple[1] |                    
                                CASE 
                                    WHEN succ IS NOT NULL AND succ.earlyStart IS NOT NULL AND succ.earlyStart < minSuccEarlyStart 
                                    THEN succ.earlyStart  
                                    ELSE minSuccEarlyStart 
                                END
                            ) AS minSuccEarlyStart
                        SET currentActivity += {
                            totalFloat: currentActivity.lateFinish - currentActivity.earlyFinish,
                            freeFloat: COALESCE(minSuccEarlyStart - currentActivity.earlyStart - currentActivity.duration, 0)
                        }
                        RETURN currentActivity.name AS activityName, 
                            currentActivity.earlyStart AS earlyStart, 
                            currentActivity.earlyFinish AS earlyFinish,
                            currentActivity.lateStart AS lateStart,
                            currentActivity.lateFinish AS lateFinish,
                            currentActivity.totalFloat AS totalFloat,
                            currentActivity.freeFloat AS freeFloat
                        ORDER BY currentActivity.depth DESC""")
            return results.data()
    except Exception as e:
        print(e)
        raise e
df = to_dataframe(calculateSlack())
to_csv(to_dataframe(calculateSlack()), "slack.csv") 
df

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n                        MATCH (activity:ACTIVITY)\n                        OPTIONAL MATCH (activity)-[:DEPENDS_ON]->(successor:ACTIVITY)\n\n                        WITH activity, COLLECT(successor) AS successors\n\n                        OPTIONAL MATCH path = (activity)<-[:DEPENDS_ON*]-(dependent:ACTIVITY)\n                        WITH activity, successors, \n                            COALESCE(MAX(LENGTH(path)), 0) AS depth, \n                            SIZE(successors) AS outgoingDependencies\n\n                        ORDER BY depth ASC, outgoingDependencies DESC \n\n                        WITH COLLECT([activity, successors]) AS activitySuccessorsList\n                      

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n                        MATCH (activity:ACTIVITY)\n                        OPTIONAL MATCH (activity)-[:DEPENDS_ON]->(successor:ACTIVITY)\n\n                        WITH activity, COLLECT(successor) AS successors\n\n                        OPTIONAL MATCH path = (activity)<-[:DEPENDS_ON*]-(dependent:ACTIVITY)\n                        WITH activity, successors, \n                            COALESCE(MAX(LENGTH(path)), 0) AS depth, \n                            SIZE(successors) AS outgoingDependencies\n\n                        ORDER BY depth ASC, outgoingDependencies DESC \n\n                        WITH COLLECT([activity, successors]) AS activitySuccessorsList\n                      

,activityName,earlyStart,earlyFinish,lateStart,lateFinish,totalFloat,freeFloat
0,Genesis,0,0,7,7,7,0
1,A - Wall panel,0,4,3,7,3,0
2,B,4,7,4,7,0,0
3,C,4,6,10,12,6,6
4,D,7,12,7,12,0,0
5,E,7,10,13,16,6,6
6,F,12,16,12,16,0,0
7,G,16,19,16,19,0,0


### 

### Finding critical path
This are activities that cannot be delayed if the goal is to meet the project deadline

In [23]:
def criticalPath():
    try:
        with driver.session(database=NEO4J_DB) as session:
            results = session.run("""
                        // trace paths between critical activities
                        OPTIONAL MATCH path = (start:ACTIVITY)-[:DEPENDS_ON*]->(end:ACTIVITY)
                        WHERE ALL(node IN NODES(path) WHERE node.totalFloat = 0)

                        //Total duration between the critical activities
                        WITH path, 
                            REDUCE(totalDuration = 0, activity IN NODES(path) | totalDuration + activity.duration) AS projectDuration
                        ORDER BY projectDuration DESC

                        // Return the longest path and duration
                        LIMIT 1
                        RETURN [activity IN NODES(path) | activity.name] AS criticalPath, projectDuration""")
            return results.data()
    except Exception as e:
        print(e)
        raise e
to_dataframe(criticalPath())

,criticalPath,projectDuration
0,"[Genesis, A, B, D, F, G]",19


Duration can either be PERT or CPM, This values should come from industry players who agree amongst each other the standard. Target their emotion that they are part of something

### Plan for upcoming tasks
Get upcoming activities and figure out the resources required if the resources will be availble or not

In [ ]:
# 2 hops
"""MATCH (tom:Person {name:'Tom Hanks'})--{2}(colleagues:Person)
RETURN DISTINCT colleagues.name AS name, colleagues.born AS bornIn
ORDER BY bornIn
LIMIT 5"""

### Table of dependencies

In [35]:

session =  driver.session(database=NEO4J_DB) 
results = session.run("""// Match all activities and their predecessors
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)

// Collect predecessors for each activity
WITH activity, COLLECT(predecessor.name) AS predecessors

// Find the depth of each activity for topological order
OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
WITH activity, predecessors, 
    COALESCE(MAX(LENGTH(path)), 0) AS depth, 
    SIZE(predecessors) AS incomingDependencies

// Order activities by depth (descending) and by number of incoming dependencies (ascending)
ORDER BY depth DESC, incomingDependencies ASC 

// Return the results with the predecessors as a list
RETURN 
    activity.name AS name, 
    depth, 
    incomingDependencies, 
    predecessors AS predecessorNames
""")
df = to_dataframe(results.data())
df

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '// Match all activities and their predecessors\nMATCH (activity:ACTIVITY)\nOPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)\n\n// Collect predecessors for each activity\nWITH activity, COLLECT(predecessor.name) AS predecessors\n\n// Find the depth of each activity for topological order\nOPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)\nWITH activity, predecessors, \n    COALESCE(MAX(LENGTH(path)), 0) AS depth, \n    SIZE(predecessors) AS incomingDependencies\n\n// Order activities by depth (descending) and by number of incoming dependencies (ascending)\nORDER BY depth DESC, incomingDependencies ASC \n\n// Return the results with the predecessors as

,name,depth,incomingDependencies,predecessorNames
0,Genesis,5,0,[]
1,A,4,1,[Genesis]
2,B,3,1,[A]
3,C,2,1,[A]
4,D,2,1,[B]
5,E,1,1,[B]
6,F,1,2,"[C, D]"
7,G,0,2,"[E, F]"


In [ ]:
# # depends on will be the ID of the predecessor activities
# def addResourceToActivity(resource: Resource, activity: Activity):
    
#     try:
#         with driver.session(database="neo4j") as session:
#             if len(predecessors):
#                 predecessors = [{"activity": vars(obj.activity), "type": obj.type, "duration": obj.duration} for obj in predecessors] # serialize the objects
#                 # activity has predecessors
#                 session.run("""
#                     CREATE (activity:ACTIVITY {name: $name, description: $description, duration: $duration})
#                     WITH activity, $predecessors as batch
#                     UNWIND batch as dependency
#                     MATCH (predecessor:ACTIVITY)
#                     WHERE ID(predecessor) = dependency.activity.id
#                     CREATE (activity)<-[:DEPENDS_ON {type: dependency.type, duration: dependency.duration}]-(predecessor)""", predecessors=predecessors, name=activity.name, description=activity.description, duration=activity.duration)

#             else:
#                 # Activity has no predecessor start nodes
#                 session.run("""
#                         MATCH (genesis:ACTIVITY {name: "Genesis"})
#                         CREATE (start:ACTIVITY {name: $name, description: $description, duration: $duration, earlyStart: COALESCE(genesis.earlyFinish, 0), earlyFinish: COALESCE(genesis.earlyFinish, 0) + COALESCE($duration, 0)})
#                         CREATE (start)<-[:DEPENDS_ON]-(genesis)
#                     """, 
#                     name = activity.name,
#                     description = activity.description,
#                     duration = activity.duration)
#     except Exception as e:
#         raise e

In [ ]:
from neo4j import GraphDatabase
from pyvis.network import Network


def fetch_network_data():
    query = """
    MATCH (a:ACTIVITY)-[r:DEPENDS_ON]->(b:ACTIVITY)
    RETURN a, b, r
    """
    with driver.session() as session:
        result = session.run(query)
        return [
            (record["a"], record["b"], record["r"])
            for record in result
        ]

# Fetch and visualize
data = fetch_network_data()



In [ ]:
# used to plot the network digram
data